In [ ]:
import pandas as pd 
import csv


## TODO: SET CURRENT ITERATION
## REQUIRES qc_output.csv AND MASTER of prev iteration
iteration = 5





file = pd.read_csv("qc_output.csv")
master_data = pd.read_csv(f'MASTER_{iteration - 1}.csv')

hit_results = dict()
prompt_info = dict()

decision_indices = dict()

headings = list(file.columns.values)
STORY_ID_INDEX = headings.index("Input.Referring_Story_Id")
REFERRING_DECISION_INDEX = headings.index("Input.Referring_Decision")

INPUT1_ID_INDEX = headings.index("Input.Input_1_Id")
INPUT2_ID_INDEX = headings.index("Input.Input_2_Id")
INPUT3_ID_INDEX = headings.index("Input.Input_3_Id")
INPUT4_ID_INDEX = headings.index("Input.Input_4_Id")
INPUT1_INDEX = headings.index("Input.Input_1")
INPUT2_INDEX = headings.index("Input.Input_2")
INPUT3_INDEX = headings.index("Input.Input_3")
INPUT4_INDEX = headings.index("Input.Input_4")
INPUT1_SENSE_INDEX = headings.index("Answer.Input_1_sense")
INPUT2_SENSE_INDEX = headings.index("Answer.Input_2_sense")
INPUT3_SENSE_INDEX = headings.index("Answer.Input_3_sense")
INPUT4_SENSE_INDEX = headings.index("Answer.Input_4_sense")
INPUT1_INTEREST_INDEX = headings.index("Answer.Input_1_interesting")
INPUT2_INTEREST_INDEX = headings.index("Answer.Input_2_interesting")
INPUT3_INTEREST_INDEX = headings.index("Answer.Input_3_interesting")
INPUT4_INTEREST_INDEX = headings.index("Answer.Input_4_interesting")
NEGQC1_INDEX = headings.index("Answer.neg_qual_1_sense")
NEGQC2_INDEX = headings.index("Answer.neg_qual_2_sense")
POSQC_INDEX = headings.index("Answer.pos_qual_sense")


class QC_Branch_Data: 

  def __init__(self, row):
    self.story_id = row[STORY_ID_INDEX]
    self.ref_decision = decision_indices[row[REFERRING_DECISION_INDEX]]
    self.input_ids = row[INPUT1_ID_INDEX:INPUT1_ID_INDEX + 4]
    self.input_content = row[INPUT1_INDEX:INPUT1_INDEX + 4]


class QC_Prompt_Data:
  def __init__(self, index):
    self.index = index
    self.yes_count = 0
    self.scores = 0
    self.count = 0
  

  def new_entry(self, sense, interest):
    sense = str(sense)
    interest = int(interest)
    self.yes_count += (1 if sense == "Yes" else 0)
    self.scores += interest
    self.count += 1


for row in master_data.iterrows(): 
  decision_indices[row[1]["Decision1"]] = 1
  decision_indices[row[1]["Decision2"]] = 2 

# GOING THROUGH ROWS TO GET EACH RESPONSE
for index, row in enumerate(file.iterrows()): 
  
  row = row[1]
  story_data = QC_Branch_Data(row)

  # IF TURKER FAILS GOLD STANDARD ANSWERS
  if row[POSQC_INDEX] is "No" or (row[NEGQC1_INDEX] == "Yes" and row[NEGQC2_INDEX] == "Yes"):
    continue
  else:
    id = (story_data.story_id, story_data.ref_decision)
    if id in prompt_info: 
      for i in range(4):
        sense_index = headings.index(f"Answer.Input_{i + 1}_sense")
        interest_index = headings.index(f"Answer.Input_{i + 1}_interesting")

        sense = row[sense_index]
        interest = row[interest_index]

        hit_results[id][i].new_entry(sense, interest)
    else:
      prompt_info[id] = story_data
      hit_results[id] = [QC_Prompt_Data(i) for i in range(4)]


final_results = []

def key_func(x):
  return x.yes_count * 100 + x.scores

for id in hit_results:
  # GETTING BEST RESPONSE FOR BRANCH
  data = hit_results[id]
  data = max(data, key=lambda x : key_func(x))

  # PARSING STORY FOR DECISIONS
  story, decisions = prompt_info[id].input_content[data.index].split("\\nOption 1: ")
  dec1, dec2 = decisions.split("\\nOption 2: ")

  final_results += [(id[0], id[1], prompt_info[id].input_ids[data.index], story, dec1, dec2)]

# WRITING RESULTS TO CSV
with open('QC_MODULE_OUTPUT.csv', mode='w') as csvfile:
      writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
      writer.writerow(["Referring_Story_Id",
                        "Referring_Decision",
                        "HitId",
                        "Input",
                        "Decision_1",
                        "Decision_2"])

      for l in final_results:
          writer.writerow(l)
    

In [290]:
# QC MODULE OUTPUT TO MASTER

new_master_data = None

qc_data = pd.read_csv('QC_MODULE_OUTPUT.csv')
story_cons = dict() #(ref_story, ref_decision) -> (HitId, input, decision1, decision2)


for i, row in qc_data.iterrows():
  ref_story = row["Referring_Story_Id"]
  ref_dec = int(row["Referring_Decision"])
  hit_id = row["HitId"]
  input = row["Input"]
  dec_1 = row["Decision_1"]
  dec_2 = row["Decision_2"]

  story_cons[(ref_story, ref_dec)] = (hit_id, input, dec_1, dec_2)

new_master_data = df = pd.DataFrame({
                     "HitId":[],
                     "Text":[], 
                     "Decision1": [],
                     "Decision2": [],
                     "Decision1_HitId": [],
                     "Decision2_HitId": [],
                     "Referring_Story_Id": [],
                     "Referring_Decision": []
                     })

for i, row in master_data.iterrows():
  ref_story = row["Referring_Story_Id"]
  ref_dec = row["Referring_Decision"]
  hit_id = row["HitId"]
  input = row["Text"]
  dec_1 = row["Decision1"]
  dec_2 = row["Decision2"]
  dec_1_id = story_cons[(hit_id, 1)][0] if (hit_id, 1) in story_cons else row["Decision1_HitId"]
  dec_2_id = story_cons[(hit_id, 2)][0] if (hit_id, 2) in story_cons else row["Decision2_HitId"]

  df = pd.DataFrame({"HitId":[hit_id],
                     "Text":[input], 
                     "Decision1": [dec_1],
                     "Decision2": [dec_2],
                     "Decision1_HitId": [dec_1_id],
                     "Decision2_HitId": [dec_2_id],
                     "Referring_Story_Id": ["N/A" if hit_id == "ORIGINAL" else ref_story],
                     "Referring_Decision": ["N/A" if hit_id == "ORIGINAL" else ("1" if ref_dec == 1.0 else "2") ]
                     })
  

  new_master_data = new_master_data.append(df, ignore_index=True)
                     

# Add new data to master_data dataframe
for (ref_story, ref_dec), (hit_id, input, dec_1, dec_2) in story_cons.items():
  df = pd.DataFrame({"HitId":[hit_id],
                     "Text":[input], 
                     "Decision1": [dec_1],
                     "Decision2": [dec_2],
                     "Decision1_HitId": [""],
                     "Decision2_HitId": [""],
                     "Referring_Story_Id": [ref_story],
                     "Referring_Decision": [ref_dec]
                     })
  

  new_master_data = new_master_data.append(df, ignore_index=True)

new_master_data.to_csv(f'MASTER_{iteration}.csv', float_format='{:,.0f}', index=False)  